one-hot encoding 을 NLP 에? 



# Text => NLP 의 시작 

NLP 에 대해서 이제 얘기를 하게 될 것이다. 

글자에 넘버링을 주는 것이 아니라, 단어에 넘버링을 줘서 학습을 시킬 예정이다. 

원래는 아스키 코드를 이용해서 하려고 했었으나, silent, listen 과 같이 완벽하게 동일한 글자를 사용하는 것이 있기 때문에 그렇게는 하지 않을 것이라고 한다. 




# Tokenizer

##### tokenizer(num_words =100) 

num_words 라는 hyperparameter 를 통해서 갖고 있는 문장 데이터 내에서 사용 빈도수가 제일 많은 100개의 단어들을 갖고 encode 를 할 것이라고 한다. 

#### tokenizer.fit_on_texts(sentences) 


fit_on_texts 라는 메소드를 이용해서 sentences 를 받을 것이고 encode 를 진행하는 것이다. 

#### word_index = tokenizer.word_index

이로 인해서 단어들이 token 화로 (encoding) 이 되는 것을 볼수 잇다. 

여기서 나타나는 특징 중 하나가, tokenizer 를 하게 되면 대문자 I 도 소문자 i 로 인식하게 된다고 한다. 

tokenizer provides a word index property which returns a dictionary containing key value pairs, where the key is the 

word, and the value is the token for that word, which you can inspect by simply printing it out. 

#### 만약에 dog! 이 붙게 된다면 어떻게 될까? 
! 로 인해서 dog! 이라는 토큰이 생기지는 않는다. 그냥 dog 로 인식하게 되는 것이다. 



In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
    'i love my dog',
    'I, love my cat',
    'You love my dog!', 
    'me love your dog',
    'yes, I love my baby!'
]

tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'me': 7, 'yes': 9, 'love': 1, 'i': 3, 'dog': 4, 'cat': 5, 'baby': 10, 'you': 6, 'my': 2, 'your': 8}


text corpus : 말뭉치 


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = ['I love my dog',
            'I love my. at',
            'You love my dog!',
            'do you think my dog is amazing? ']

tokenizer = Tokenizer(num_words =100)
tokenizer.fit_on_texts(sentences)
word_index =tokenizer.word_index
sequences =tokenizer.texts_to_sequences(sentences)

print(word_index)
print("\n")
print(sequences)

{'do': 7, 'love': 2, 'i': 4, 'is': 9, 'dog': 3, 'amazing': 10, 'at': 6, 'you': 5, 'my': 1, 'think': 8}


[[4, 2, 1, 3], [4, 2, 1, 6], [5, 2, 1, 3], [7, 5, 8, 1, 3, 9, 10]]


##### OOV 이란? out-of-vocabulary

https://omicro03.medium.com/자연어처리-nlp-5일차-단어-분리-60b59f681eb7

예시에서 OOV Token 이라는 것을 사용하고 있다. OOV -> out of vocabulary 의 약자이다. 

단어 분리 (subword Segmnetation ) 이라는 것은 기계가 아직 배운적이 없는 단어더라도 마치 배운 것처럼 대처할 수 있도록 도와주는 기법 중 하나라고 한다. 이 방법은 이제 기계 번역 등 에서 주요 전처리로 사용되고 있다. 

즉, 기계 가 모르는 단어에 대처하지 못하는 상황이 오면 이 상황을 OOV 문제라고 언급한다. 

이제 단어분리를 통해 OOV 문제를 해결할 수 있는 이유와, 대표적인 단어 분리 방법인 BPE(Byte Pair Encoding) 알고리즘에 대해서 학습한다고 한다. 

단어 분리 ( subword segmentation ) 이라는 것이 이유있는 이유는, 하나의 단어는 의미있는 여러 단어의 조합으로 구성된 경우가 많기 때문에, 단어를 여러 단어로 분리 해 보겠다는 전처리 작업이다. 

WPM( word piece model ) 을 보면 뭐 하는 것인지 정확하게 감은 오지 않는다. 

Jet makers feud over seat width with big orders at stake

_J et _makers _fe ud _over _seat _width _with _big _orders _at _stake



WPM이 수행된 결과로부터 다시 수행 전의 결과로 돌리는 방법은 현재 있는 띄어쓰기를 전부 삭제하여 내부 단어들을 다시 하나의 단어로 연결시키고, 언더바를 다시 띄어쓰기로 바꾸면 된다.


BPE (Byte Pair Encoding) 

BPE 알고리즘의 기본 원리는 가장 많이 등장한 문자열에 대하여 병합하는 작업을 반복하는 데, 원하는 단어 집합의 크기. 즉, 단어의 갯수가 될 때까지 이 작업을 반복한다.


## 어떻게 작동하는지 

1. 우린는 상당히 큰 양의 단어 데이터가 필요하다. 
2. 보지 못했었던 단어들을 그냥 무시하는 것 보다는, unseen word 를 보게 되면 special value 를 부여하게 하여 코드를 ㅅ짤수 있게 된다. 

예를들면 training data 로는 

i really love my dog (love 에 주목)  

이라고 되어있는데 학습 데이터로 존재 하지 않으니, OOV token 이라는 것은 이런식으로 작동을 한다. 

test data = i really love my dog

i OOV love my dog 

뭐 이런 식으로 대충 들어간다고 생각하면 될 것 같다. 

### pading

여기에도 padding 이 들어가게 된다. 
- image 에서는 image 사이즈가 UNiform 해야지 trianing 이 가능함을 알고 있다. 

- text 에서도 비슷하다. 우리는 some level of uniformity of size 가 필요하고, 그래서 padding 이 이제 사용될 것이라고 한다. 

tensorflow 에서 padding 관련된 것이 있다. 

- padded = pad_seqeunces(sequences) 

를 통해서 padding 이 진행 될 것이다. 

문장의 길이가 다 다르기 때문에 이것을 맞춰주는 작업을 padding 이 해 주는 것이다 

0,0,0,5,3,2,4

0,0,0,5,3,2,7

0,0,0,6,4,3,2

8,6,7,23,5,3,4

이런 식으로 진행하는 것이다. 길이를 맞춰서 0 을 넣는 것이라고 생각하면 되는 것이다. 

- padded = pad_sequences(sequences, padding = 'post', maxlen =5 ) 

padding 하는 작업이 sentence 이후에 나올수 있을 것이다. 만약에 이후에 padding 하는 것이 좋다고 생각하면, 이런식으로 코드를 바꾸면 된다고 한다. post 라는 것을 사용하여 padding 을 sentence 이후에 할 수 잇게 하는 것이라고 한다. (정확하게 sentence 이후에 padding 을 한다는 것이 무슨 말인지 모르겠음)----> 아 000 을 넣는 것을 문장 앞에다가 할수도 있고, 뒤에다가도 할수 있다고 말하는 것이다.  
 
 maxlen 을 통해서 matrix width 를 정할 수 있게 해 줄 수 잇다. 5개의 word 를 설정하고 싶으면 이렇게 한다고 한다. 
 
####  아니 근데, 만약에 maxlenth 보다 문장 길이가 크게 된다면 어떻게 되는 것이지 ? 

문장길이가 길어지게 된다면 **문장의 앞에서부터 문장의 길이를 측정하게 되는 것이라고 한다. 

## 이제 우리는 단어의 Indexing, OOV, Padding 에 대해서 배웠다. 

그러나 문장 자체가 너무 쉽게 되어있는 것들만 사용했기 때문에 이제 좀 어려운 단어들에 대해서 공부를 해 볼 것이다. 





In [14]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# pad_sequence can use sentences in different length

sentences = [
        'I love my dog',
        'I love my cat',
        'You love my dog!',
        'Do you think my dog is amazing?'
]
# 위에 3개랑 마지막에 있는 것이랑 길이가 다름. 그래서 Pad_sequence 를 통해서 padding d
# 사용할 것이라고 한다. 
tokenizer = Tokenizer(num_words = 100, oov_token = "<OOV>")
tokenizer.fit_on_texts(sentences)
# sentence 를 갖고 word index 를 생성하고 tokenizer 를 초기화 시키는 역할을 하게 될 것이다. 
word_index = tokenizer.word_index
# word_index 를 이를 통해서 볼 것이라고 한다. vector 화 되어 있는 것을 확인하는 것이라고 
#생각하면되는 것이다. 



sequences = tokenizer.texts_to_sequences(sentences)

padded = pad_sequences(sequences, maxlen = 5)
print("\nWord index = ", word_index)
print("\nSequences = ", sequences)
# sequence 를 통해서 문장에 있는 단어들이 
print("\nPadded Sequences: ")
print(padded)

# try this words that the tokenizer wasn't fit to 

test_data = ['i really love my dog', 'my dog loves my manatee']

test_seq = tokenizer.texts_to_sequences(test_data)
print("\n test Sequences = ", test_seq)

padded = pad_sequences (test_seq, maxlen = 10)
print("\n Padded test Sequence: ")
print(padded)

('\nWord index = ', {'do': 8, 'love': 3, 'i': 5, 'is': 10, 'dog': 4, 'cat': 7, 'amazing': 11, 'you': 6, 'my': 2, 'think': 9, '<OOV>': 1})
('\nSequences = ', [[5, 3, 2, 4], [5, 3, 2, 7], [6, 3, 2, 4], [8, 6, 9, 2, 4, 10, 11]])

Padded Sequences: 
[[ 0  5  3  2  4]
 [ 0  5  3  2  7]
 [ 0  6  3  2  4]
 [ 9  2  4 10 11]]
('\n test Sequences = ', [[5, 1, 3, 2, 4], [2, 4, 1, 2, 1]])

 Padded test Sequence: 
[[0 0 0 0 0 5 1 3 2 4]
 [0 0 0 0 0 2 4 1 2 1]]


### 그런데 oov 들을 이렇게 다 1 으로만 해 두면 좋은 것일까? 난 안좋을거 같은데.. 


여태까지는 Hard coded 되어있는 것을 이용해서 진행이 되었다. 이제는 더 많은 데이터를 이용하게 될 것이다. 

public-data set 을 보자. 

sarcasm detection 을 이용해 볼 것이라고 한다. 

3개의 label 이 존재 한다. 

- 1 은 sarcastic (0/1 로 표현이 된다고 한다. )  2 headline, 3 link to the headline 

데이터를 더 쉽게 control 하기 위헤서 뭔가 바꿨다고 한다. 

우리는 json 을 이용해서 뭔가 조작할 것이라고 한다. 

import json

이제 json 형태를 python 에서 사용할 수 있게 해줄 것이라고 한다. 

with open("sarcasm.json:, 'r') as f:

json 열어. 그리고 load 리스트 

datastore = json.load(f)

sentences = []

labels = []

urls = []

senteces, label, urls 를 list 의 형태로 tokenized 되어잇는 것을 넣을 것이기 때문에 이렇게 했다고 한다. 

여아튼 다 list 형식으로 한다고 한다. 

open 




In [17]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json
  
import json
with open ("/tmp/sarcasm.json", 'r') as f :
    datastore = json.load(f)
    
sentences = []
labels = []
urls = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])
    
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer  = Tokenizer (oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
print(len(word_index))
print(word_index)

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding = 'post')
print(padded[0])
print(padded.shape)

--2021-08-03 23:01:53--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 34.64.4.48, 34.64.4.16, 34.64.4.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|34.64.4.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: `/tmp/sarcasm.json'

/tmp/sarcasm.json   100%[===================>]   5.38M  2.19MB/s    in 2.5s    

2021-08-03 23:01:56 (2.19 MB/s) - `/tmp/sarcasm.json' saved [5643545/5643545]

29657
{u'schlegel': 20584, u"considered'": 25979, u'woods': 3738, u'spiders': 21797, u'hanging': 4423, u'woody': 3668, u'scold': 15377, u'rickman': 10498, u"wood'": 27465, u'eugenics': 25367, u'appropriation': 25683, u"grandson'": 17808, u'broward': 28789, u'bringing': 2555, u"storybook'": 28407, u'tcby': 20525, u'unpauses': 19444, u'wooded': 21214, u'wooden': 5557, u'wednesday': 4534, u'circuitry': 24228, u'crotch': 684

[  308 15115   679  3337  2298    48   382  2576 15116     6  2577  8434
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
(26709, 40)


What is the name of the method used to tokenize a list of sentences?
- fit_on_texts(sentneces)

Once you have the corpus tokenized, what’s the method used to encode a list of sentences to use those tokens?
- texts_to_tokens(sentences)

# encode 랑 tokenize 의 차이가 뭐지 ? 
If you don’t use a token for out of vocabulary words, what happens at encoding?

-  The word isn’t encoded, and is skipped in the sequence


